## char-CNN emotion analysis

In [1]:
import pandas as pd
import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import vocab
from torchtext import data
from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data import Iterator
from torchtext.vocab import GloVe
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pickle
import dill
import glob

In [2]:
batch_size = 20

In [3]:
def tokenizer(x):
    return x.split()

In [4]:
path = "../data/text_emotion.csv"
allFiles = glob.glob(path)
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, sep=',')
    list_.append(df)
dataset = pd.concat(list_, axis = 0, ignore_index = True)
dataset[:10]

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?


In [5]:
BOS_WORD = '<s>'
EOS_WORD = '</s>'
BLANK_WORD = "<blank>"
max_vocab = 8000
fix_length=30

In [6]:
CHAR_NESTING = Field(sequential=True, tokenize=lambda x: list(x), pad_token=BLANK_WORD, lower=True, batch_first=True, fix_length=fix_length)
CHAR = data.NestedField(CHAR_NESTING)
WORD_NESTING = Field(sequential=True, tokenize=lambda x: x.split(), pad_token=BLANK_WORD, lower=True, batch_first=True, fix_length=fix_length)
WORD = data.NestedField(WORD_NESTING)
LABEL = Field(sequential=False, unk_token=None, tokenize=lambda x: x)

In [7]:
#CHAR = Field(sequential=True, tokenize=lambda x: list(x), pad_token=BLANK_WORD, lower=True, batch_first=True, fix_length=fix_length)
#LABEL = Field(sequential=False, unk_token=None, tokenize=lambda x: x)

In [8]:
#https://www.crowdflower.com/wp-content/uploads/2016/07/text_emotion.csv
train_data = TabularDataset(path='../data/text_emotion.csv', 
                            format='csv', 
                            skip_header=True,
                            fields=[("tweet_id", None),("sentiment", LABEL),("author", None),(("word", "char"),(WORD, CHAR))])

In [9]:
#glove = vocab.Vectors('../data/glove.6B.300d.txt')
#tqdm_notebook().pandas() 
#https://medium.com/@sonicboom8/sentiment-analysis-torchtext-55fb57b1fab8

In [10]:
#TEXT.build_vocab(train_data, max_size=max_vocab)
WORD.build_vocab(train_data.word)
CHAR.build_vocab(train_data.char, min_freq=3)
LABEL.build_vocab(train_data)

In [11]:
CHAR.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<blank>': 1,
             'e': 2,
             't': 3,
             'o': 4,
             'a': 5,
             'i': 6,
             'n': 7,
             's': 8,
             'r': 9,
             'h': 10,
             'l': 11,
             'd': 12,
             'm': 13,
             'u': 14,
             'y': 15,
             'g': 16,
             'w': 17,
             'c': 18,
             '.': 19,
             'p': 20,
             'f': 21,
             'b': 22,
             'k': 23,
             'v': 24,
             '!': 25,
             '@': 26,
             "'": 27,
             ',': 28,
             'j': 29,
             '?': 30,
             '/': 31,
             'x': 32,
             'z': 33,
             ';': 34,
             '-': 35,
             '&': 36,
             ':': 37,
             'q': 38,
             '1': 39,
             '2': 40,
             '0': 41,
             '3

In [12]:
one_example = train_data.examples[0]
print(one_example.word)
print(one_example.char)

[['@tiffanylue'], ['i'], ['know'], ['i'], ['was'], ['listenin'], ['to'], ['bad'], ['habit'], ['earlier'], ['and'], ['i'], ['started'], ['freakin'], ['at'], ['his'], ['part'], ['=[']]
[['@', 't', 'i', 'f', 'f', 'a', 'n', 'y', 'l', 'u', 'e'], ['i'], ['k', 'n', 'o', 'w'], ['i'], ['w', 'a', 's'], ['l', 'i', 's', 't', 'e', 'n', 'i', 'n'], ['t', 'o'], ['b', 'a', 'd'], ['h', 'a', 'b', 'i', 't'], ['e', 'a', 'r', 'l', 'i', 'e', 'r'], ['a', 'n', 'd'], ['i'], ['s', 't', 'a', 'r', 't', 'e', 'd'], ['f', 'r', 'e', 'a', 'k', 'i', 'n'], ['a', 't'], ['h', 'i', 's'], ['p', 'a', 'r', 't'], ['=', '[']]


In [13]:
train_loader = Iterator(train_data, 
                        batch_size=batch_size, 
                        repeat=False)

In [14]:
for batch in train_loader:
    break;
#print(batch.word.shape)    
#print(batch.word[:,:3,:4])
print(batch.char.shape)
#print(batch.char[:,:3,:6])
print(batch.sentiment.shape)

torch.Size([20, 24, 30])
torch.Size([20])


In [15]:
#TEXT.vocab.stoi

In [16]:
len(train_data)

40000

In [17]:
CHAR.vocab.itos[4]

'o'

In [18]:
with open("model/CHAR2.Field","wb")as f:
     dill.dump(CHAR,f)

In [19]:
with open("model/LABEL_CHAR2.Field","wb")as f2:
     dill.dump(LABEL,f2)

In [20]:
def init_network(model, method='xavier', exclude='embedding', seed=123):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    for name, w in model.named_parameters():
        if not exclude in name:
            if 'weight' in name:
                if method is 'xavier':
                    nn.init.xavier_normal_(w)
                elif method is 'kaiming':
                    nn.init.kaiming_normal_(w)
                else:
                    nn.init.normal_(w)
            elif 'bias' in name:
                nn.init.constant_(w, 0.0)
            else: 
                pass

In [21]:
def print_model(model, ignore='embedding'):
    total = 0
    for name, w in model.named_parameters():
        if not ignore or ignore not in name:
            total += w.nelement()
            print('{} : {}  {} parameters'.format(name, w.shape, w.nelement()))
    print('-------'*4)
    print('Total {} parameters'.format(total))

In [22]:
with open("model/CHAR2.Field","rb")as f:
     CHAR=dill.load(f)
        
with open("model/LABEL_CHAR2.Field","rb")as f2:
     LABEL=dill.load(f2)

In [23]:
'''
batch_size=64
epochs=200
embidding_dim = 300
seq_length = 50
vocab_size = len(TEXT.vocab.itos)
num_filters = 128
kernel_sizes = [1,2,3,4,56,]
hidden_dim = 128 # hidden size of fully conntected layer
label_size = len(LABEL.vocab)
print_every = 1000
'''

'\nbatch_size=64\nepochs=200\nembidding_dim = 300\nseq_length = 50\nvocab_size = len(TEXT.vocab.itos)\nnum_filters = 128\nkernel_sizes = [1,2,3,4,56,]\nhidden_dim = 128 # hidden size of fully conntected layer\nlabel_size = len(LABEL.vocab)\nprint_every = 1000\n'

In [24]:
char_vocab_size = len(CHAR.vocab.itos)
char_embed_dim = 15
label_size = len(LABEL.vocab.itos)
#word_vocab_size = len(LABEL.vocab.itos)
#word_embed_dim = 256

kernel_widths = [1, 2, 3, 4, 5, 6]
kernel_sizes = [25, 50, 75, 100, 125, 150] #525
dropout_prob = 0.5

rnn_hidden = 256
lstm_num_layers = 2

param_init = 0.05
learning_rate_decay = 0.5
decay_when = 1.0
learning_rate = 1.0


max_epoch = 200
max_steps = 10000
max_sent_len = 35
max_word_len = 30
clip = 5.0

In [25]:
## TODO
class Highway(nn.Module):
    def __init__(self, input_size):
        super(Highway, self).__init__()
        self.fc1 = nn.Linear(input_size, input_size, bias=True)
        self.fc2 = nn.Linear(input_size, input_size, bias=True)

    def forward(self, x):
        """
        G = relu(x, Wg)
        T = sigmoid(x, Wt)
                                   |x, T == 0
        y = G * T + x * (1. - T) = |
                                   |G, T == 1
        """
        #print(x.shape)
        t = F.sigmoid(self.fc1(x))
        return torch.mul(t, F.relu(self.fc2(x))) + torch.mul(1-t, x)

In [26]:
class HighwayNetwork(nn.Module):
    def __init__(self, input_size,activation='ReLU'):
        super(HighwayNetwork, self).__init__()
        #transform gate
        self.trans_gate = nn.Sequential(
            nn.Linear(input_size,input_size),
            nn.Sigmoid())
        
        #highway
        if activation== 'ReLU':
            self.activation = nn.ReLU()
            
        self.h_layer = nn.Sequential(
            nn.Linear(input_size,input_size),
            self.activation)
        #self.trans_gate[0].weight.data.uniform_(-0.05,0.05)
        #self.h_layer[0].weight.data.uniform_(-0.05,0.05)
        self.trans_gate[0].bias.data.fill_(-2)
        #self.h_layer[0].bias.data.fill_(0)

    def forward(self,x):

        t = self.trans_gate(x)
        h = self.h_layer(x)

        z = torch.mul(t,h)+torch.mul(1-t,x)

        return z

In [27]:
class CharCNN(nn.Module):
    #num_filters = out-channels
    def __init__(self, char_vocab, embed_dim, kernel_sizes, kernel_widths):
        super(CharCNN, self).__init__()
        
        self.kernel_sizes = kernel_sizes
        self.kernel_widths = kernel_widths
        
        self.kernel = list(zip(kernel_sizes, kernel_widths)) 
        self.convs = nn.ModuleList([nn.Conv2d(1, out_channel, 
                                              kernel_size=(kernel_widths, embed_dim))  # (height, width)
                                      for out_channel, kernel_widths in self.kernel])

    
    def forward(self, x):
        pooled = []
        for i, conv in enumerate(self.convs):
            conved = F.tanh(conv(x).squeeze(3))
            #pool  = F.max_pool2d(conved, max_word_len-kernel_widths[i],1)
            pool = F.max_pool1d(conved, conved.shape[2]).squeeze(2)
            pooled.append(pool)
        
        cat = torch.cat(pooled, 1)
        
        return cat

In [28]:
class CharCNN_LSTM(nn.Module):
    def __init__(self, char_vocab, embed_dim, hidden_dim, label_size,
                 kernel_sizes, kernel_widths, lstm_num_layers, dropout_prob):
        super(CharCNN_LSTM, self).__init__()
        
        self.char_vocab = char_vocab
        self.embed_dim = embed_dim
        self.lstm_num_layers = lstm_num_layers
        self.hidden_dim = hidden_dim
        
        #cnn layer
        self.char_embed = nn.Embedding(char_vocab, embed_dim, padding_idx=1)
        self.conv = CharCNN(char_vocab, embed_dim, kernel_sizes, kernel_widths)
        
        # highway lyaer
        self.highway_input_dim = sum([x for x in kernel_sizes])
        #self.highway1 = Highway(self.highway_input_dim)
        #self.highway2 = Highway(self.highway_input_dim)
        self.highway1 = HighwayNetwork(self.highway_input_dim)
        self.highway2 = HighwayNetwork(self.highway_input_dim)
        #self.batch_norm = nn.BatchNorm1d(self.highway_input_dim, affine=False)
        
        
        # LSTM
        self.lstm = nn.LSTM(self.highway_input_dim, 
                            self.hidden_dim, 
                            lstm_num_layers,
                            batch_first=True,
                            dropout = dropout_prob)
        
        # output layer
        self.drop = nn.Dropout(dropout_prob)
        self.softmax = nn.LogSoftmax()
        self.linear = nn.Linear(self.hidden_dim, label_size)
        self.hidden = None

    def forward(self, x, h):
        this_batch_size = x.size()[0]
        this_seq_len = x.size()[1]
        #max_word_len = x.size()[2]
        max_wlen = max_word_len
        
        x = x.contiguous().view(-1, x.size()[2])
        #x = x.contiguous().view(-1, max_wlen)

        emb = self.char_embed(x)
        #emb = emb.view(x.shape[0],1,x.shape[1],x.shape[2])
        #emb = emb.view(-1, self.char_embed_dim)
        emb = emb.unsqueeze(1)

        cnn = self.conv(emb)
        
        #cnn = self.batch_norm(cnn)

        h_ = self.highway1(cnn) 
        h_ = self.highway2(h_)
        
        #h_ = h_.view(batch_size, -1, self.highway_input_dim)
        #h_ = h_.contiguous().view(batch_size, -1, self.highway_input_dim)
        h_ = h_.contiguous().view(this_batch_size,this_seq_len, -1)
        
        #print("h_ : ", h_.shape)
        #print("hidden : ", hidden[0].shape)
        #self.hidden = self.init_hidden(this_batch_size)
        
        output, h = self.lstm(h_, h)
        #print("out : ", output.shape)
        
        out = output.contiguous().view(this_batch_size, this_seq_len, -1)
        #output = self.drop(output)
        

        #decoded = output[:,-1]
        #print(out.shape)
        out = self.linear(out[:, -1])
        #print(out.shape)
        y = self.softmax(out)
        #print(y.shape)
        return y, h
    
    def init_hidden(self, batch_size=1):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (Variable(weight.new(self.lstm_num_layers, batch_size, self.hidden_dim).zero_()).cuda(),
                  Variable(weight.new(self.lstm_num_layers, batch_size, self.hidden_dim).zero_()).cuda())
        else:
            hidden = (Variable(weight.new(self.lstm_num_layers, batch_size, self.hidden_dim).zero_()),
                      Variable(weight.new(self.lstm_num_layers, batch_size, self.hidden_dim).zero_()))
        return hidden



In [29]:
# GPU 사용할 수 있는지 확인
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')
#train_on_gpu = False

Training on GPU!


In [30]:
def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
        return Variable(x, volatile=volatile)

In [31]:
model = CharCNN_LSTM(char_vocab_size, char_embed_dim, rnn_hidden, label_size,
                     kernel_sizes, kernel_widths, lstm_num_layers, dropout_prob)
init_network(model)
if(train_on_gpu):
    model.cuda()
    
#hidden_state = model.init_hidden(batch_size)
#model.hidden = model.init_hidden(batch_size)

hidden_state = (to_var(torch.zeros(2,batch_size,rnn_hidden)),to_var(torch.zeros(2,batch_size,rnn_hidden)))
    

model.train()

CharCNN_LSTM(
  (char_embed): Embedding(71, 15, padding_idx=1)
  (conv): CharCNN(
    (convs): ModuleList(
      (0): Conv2d(1, 25, kernel_size=(1, 15), stride=(1, 1))
      (1): Conv2d(1, 50, kernel_size=(2, 15), stride=(1, 1))
      (2): Conv2d(1, 75, kernel_size=(3, 15), stride=(1, 1))
      (3): Conv2d(1, 100, kernel_size=(4, 15), stride=(1, 1))
      (4): Conv2d(1, 125, kernel_size=(5, 15), stride=(1, 1))
      (5): Conv2d(1, 150, kernel_size=(6, 15), stride=(1, 1))
    )
  )
  (highway1): HighwayNetwork(
    (trans_gate): Sequential(
      (0): Linear(in_features=525, out_features=525, bias=True)
      (1): Sigmoid()
    )
    (activation): ReLU()
    (h_layer): Sequential(
      (0): Linear(in_features=525, out_features=525, bias=True)
      (1): ReLU()
    )
  )
  (highway2): HighwayNetwork(
    (trans_gate): Sequential(
      (0): Linear(in_features=525, out_features=525, bias=True)
      (1): Sigmoid()
    )
    (activation): ReLU()
    (h_layer): Sequential(
      (0): Linea

In [32]:
#criterion = F.cross_entropy
#criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss()
#criterion = nn.BCEWithLogitsLoss()
#optimizer = optim.Adam(model.parameters(), lr=1e-3)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3,momentum=0.8)
#optimizer = torch.optim.SGD(model.parameters(), lr=1.0)
optimizer = torch.optim.Adam(model.parameters())
#optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.85)
print_model(model, ignore=None)

char_embed.weight : torch.Size([71, 15])  1065 parameters
conv.convs.0.weight : torch.Size([25, 1, 1, 15])  375 parameters
conv.convs.0.bias : torch.Size([25])  25 parameters
conv.convs.1.weight : torch.Size([50, 1, 2, 15])  1500 parameters
conv.convs.1.bias : torch.Size([50])  50 parameters
conv.convs.2.weight : torch.Size([75, 1, 3, 15])  3375 parameters
conv.convs.2.bias : torch.Size([75])  75 parameters
conv.convs.3.weight : torch.Size([100, 1, 4, 15])  6000 parameters
conv.convs.3.bias : torch.Size([100])  100 parameters
conv.convs.4.weight : torch.Size([125, 1, 5, 15])  9375 parameters
conv.convs.4.bias : torch.Size([125])  125 parameters
conv.convs.5.weight : torch.Size([150, 1, 6, 15])  13500 parameters
conv.convs.5.bias : torch.Size([150])  150 parameters
highway1.trans_gate.0.weight : torch.Size([525, 525])  275625 parameters
highway1.trans_gate.0.bias : torch.Size([525])  525 parameters
highway1.h_layer.0.weight : torch.Size([525, 525])  275625 parameters
highway1.h_layer.0.

In [33]:
def repackage_hidden(h): # todo
    """ wraps hidden states in new Variable, to detach them from their history """
    #print(type(h))
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(Variable(v.data) for v in h)

In [34]:
epochs = 200
print_every = 100

best_acc = 0.0
counter = 0
index = 0

for e in range(epochs):
    for i,batch in enumerate(train_loader):
        counter += 1
        
        #print(batch.char.shape)
        #print(batch.sentiment.shape)
        #if len(batch) != batch_size: continue
        if(train_on_gpu):
            inputs, targets = Variable(batch.char).cuda(), Variable(batch.sentiment).cuda()
        else:
            inputs, targets = batch.char, batch.sentiment
        counter += 1
        
        #hidden_state = repackage_hidden(model.hidden)
        model.zero_grad()
        
        #output, hidden_state = model(inputs, hidden_state)
        output, hidden_state = model(inputs, hidden_state)
        #print("output: ", output.shape)
        #print("targets: ", targets.shape)
        
        #print("out : ", output.shape)
        #print(hidden_state[0].shape)
        
        #print(targets.view(-1).shape)
        targets = targets.view(batch_size)
        loss = criterion(output, targets)
        #loss = F.cross_entropy(output, targets.view(-1))
        loss.backward()
        #torch.nn.utils.clip_grad_norm(net.parameters(), 5, norm_type=2)
        optimizer.step()
        
        if counter % print_every == 0:
            print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

## model save and load

In [64]:
import os
filename = "cls_emotion_char_cnn2.pth"
PATH = os.path.join("model", filename)
torch.save(model.state_dict(), PATH)

In [76]:
#model = TextCNN(TEXT.vocab.vectors, TEXT.vocab.stoi[TEXT.pad_token], num_filters, kernel_sizes, label_size, 0.5)
model = TextCNN(TEXT.vocab.vectors, TEXT.vocab.stoi[TEXT.pad_token], vocab_size, embidding_dim, num_filters, kernel_sizes, label_size, 0.1)

In [77]:
model.load_state_dict(torch.load(PATH))

In [226]:
#sentence = "How are YOU convinced that I have always wanted you? What signals did I give off...damn I think I just lost another friend"
#sentence = "The storm is here and the electricity is gone"
#sentence = "Damm servers still down  i need to hit 80 before all the koxpers pass me"
#sentence = "Need to pack for CALI CALI! Cannot waittt! Thinking a glass of wine is in order to celebrate my weekend vaca. Still work 2morrow, tho."
#sentence = "I'm worried I can do anything"
##sentence = "I felt ecstatic when I passed my exam"
#sentence = "I was overjoyed at the birth of my son."
sentence = "During the Christmas holidays I felt wonderfully merry."
#sentence = "I’m feeling a little low at the moment."
#sentence = "I was so annoyed when I failed my English test."
#sentence = "Afraid of your own shadow"

In [227]:
s = [TEXT.vocab.stoi[word.lower()] for word in tokenizer(sentence)]
s

[828, 4, 3278, 1925, 2, 872, 0, 0]

In [228]:
nse = np.asarray(s)
feature_tensor = torch.from_numpy(nse)
feature_tensor = feature_tensor.unsqueeze(0)
batch_size = feature_tensor.size(0)

In [229]:
if(train_on_gpu):
    feature_tensor = feature_tensor.cuda()
    model.cuda()

model.eval()
print(feature_tensor.shape)

torch.Size([1, 8])


In [230]:
output = model(feature_tensor).squeeze()
output

tensor([  3.4776,   3.9867,  -1.0742,   1.3718,   2.4690,  -5.4473,  -7.9056,
        -15.8092, -10.7468,  -4.9786,  -8.9972, -10.3861, -16.7447],
       device='cuda:0', grad_fn=<SqueezeBackward0>)

In [231]:
output.type(torch.FloatTensor)

tensor([  3.4776,   3.9867,  -1.0742,   1.3718,   2.4690,  -5.4473,  -7.9056,
        -15.8092, -10.7468,  -4.9786,  -8.9972, -10.3861, -16.7447],
       grad_fn=<CopyBackwards>)

In [232]:
pred = F.softmax(output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [233]:
#pred.type(torch.FloatTensor)

In [234]:
val,idx = pred.sort(descending=True)

In [235]:
val

tensor([5.2631e-01, 3.1634e-01, 1.1538e-01, 3.8514e-02, 3.3369e-03, 6.7245e-05,
        4.2082e-05, 3.6016e-06, 1.2089e-06, 3.0147e-07, 2.1017e-07, 1.3304e-09,
        5.2203e-10], device='cuda:0', grad_fn=<SortBackward>)

In [236]:
idx

tensor([ 1,  0,  4,  3,  2,  9,  5,  6, 10, 11,  8,  7, 12], device='cuda:0')

In [246]:
value = []
for v in val.tolist():
    v = round(v, 4)
    value.append(v)
    print(v)

0.5263
0.3163
0.1154
0.0385
0.0033
0.0001
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [247]:
value[:4]

[0.5263, 0.3163, 0.1154, 0.0385]

In [248]:
emotion = []
for v in idx.tolist():
    emotion.append(LABEL.vocab.itos[v])
    print(v)

1
0
4
3
2
9
5
6
10
11
8
7
12


In [249]:
emotion[:4]

['worry', 'neutral', 'love', 'sadness']

In [259]:
result = zip(emotion, value)

In [261]:
for z in result:
    print(z)

('worry', 0.5263)
('neutral', 0.3163)
('love', 0.1154)
('sadness', 0.0385)
('happiness', 0.0033)
('empty', 0.0001)
('surprise', 0.0)
('fun', 0.0)
('enthusiasm', 0.0)
('boredom', 0.0)
('hate', 0.0)
('relief', 0.0)
('anger', 0.0)


In [74]:
'''
_, predicted = torch.max(output, 0)
value = predicted.data.tolist()
value
'''

3

In [75]:
LABEL.vocab.itos[value]

'sadness'

In [41]:
LABEL.vocab.itos

['neutral',
 'worry',
 'happiness',
 'sadness',
 'love',
 'surprise',
 'fun',
 'relief',
 'hate',
 'empty',
 'enthusiasm',
 'boredom',
 'anger']